# Projet ML

In [78]:
import pandas as pd
import numpy as np
import matplotlib as mp

# Dataset classification

Immunotherapy Dataset Data Set

lien:
https://archive.ics.uci.edu/ml/datasets/Immunotherapy+Dataset?fbclid=IwAR3UvWjj_2qqoS-_bjMEWCnrSUNS-1LezCZk1G6yMLHf_lAbpO4DZ76VZO0

In [61]:
df2 = pd.read_excel("Immunotherapy.xlsx")

In [62]:
df2.rename(
    columns={
        "sex": "Sex", 
        "age": "Age", 
        "Number_of_Warts": "Nbr_Warts",
        "induration_diameter": "Induration_diameter",
        "Result_of_Treatment": "Result", 
        
    },
    inplace = True
)

In [63]:
df2

,Sex,Age,Time,Nbr_Warts,Type,Area,Induration_diameter,Result
0,1,22,2.25,14,3,51,50,1
1,1,15,3.00,2,3,900,70,1
2,1,16,10.50,2,1,100,25,1
3,1,27,4.50,9,3,80,30,1
4,1,20,8.00,6,1,45,8,1
...,...,...,...,...,...,...,...,...
85,1,40,5.50,8,3,69,5,1
86,1,38,7.50,8,2,56,45,1
87,1,46,11.50,4,1,91,25,0
88,1,32,12.00,9,1,43,50,0


# Préprocessing

## Immunotherapy Dataset Data Set


# 1- Missing values 

Dans la description du dataset de classification c'est mentionnée qu'il y a pas de missing value
et il existe une commende qui nous permet de bien détermier si oui ou non il y a des missing values qui est:

df2 .isna().sum()  

ici df2 reprèsente notre dataset de classification

In [64]:
df2 .isna().sum()
# comme vous pouvez le remarquer tous les valeurs obtenues sont égales à zero ,par concéquent, il n'y a pas de missing values

Sex                    0
Age                    0
Time                   0
Nbr_Warts              0
Type                   0
Area                   0
Induration_diameter    0
Result                 0
dtype: int64

In [65]:
df2.dtypes

Sex                      int64
Age                      int64
Time                   float64
Nbr_Warts                int64
Type                     int64
Area                     int64
Induration_diameter      int64
Result                   int64
dtype: object

# 2- Removing Categorical values 

On a les attribut 'Sex' et 'Type' qui sont de type Categorical, il faut les transformer au type numérique:


In [66]:
df2.columns

Index(['Sex', 'Age', 'Time', 'Nbr_Warts', 'Type', 'Area',
       'Induration_diameter', 'Result'],
      dtype='object')

In [67]:
dummies_Type = pd.get_dummies(df2['Type'], prefix='Type', dummy_na=False)
dummies_Sex  = pd.get_dummies(df2['Sex'], prefix='Sex', dummy_na=False)

In [68]:
print(dummies_Type.head(5))

   Type_1  Type_2  Type_3
0       0       0       1
1       0       0       1
2       1       0       0
3       0       0       1
4       1       0       0


In [69]:
print(dummies_Sex.head(5))

   Sex_1  Sex_2
0      1      0
1      1      0
2      1      0
3      1      0
4      1      0


In [70]:
df2 = df2.drop('Sex', 1)

In [71]:
df2.head(5)

,Age,Time,Nbr_Warts,Type,Area,Induration_diameter,Result
0,22,2.25,14,3,51,50,1
1,15,3.00,2,3,900,70,1
2,16,10.50,2,1,100,25,1
3,27,4.50,9,3,80,30,1
4,20,8.00,6,1,45,8,1


In [72]:
df2 = df2.drop('Type', 1)

In [73]:
df2.head(5)

,Age,Time,Nbr_Warts,Area,Induration_diameter,Result
0,22,2.25,14,51,50,1
1,15,3.00,2,900,70,1
2,16,10.50,2,100,25,1
3,27,4.50,9,80,30,1
4,20,8.00,6,45,8,1


In [76]:
df2 = pd.concat([dummies_Sex, df2.Age, df2.Time, df2.Nbr_Warts, dummies_Type, df2.Area, df2.Induration_diameter, df2.Result], axis=1)

In [77]:
df2

,Sex_1,Sex_2,Age,Time,Nbr_Warts,Type_1,Type_2,Type_3,Area,Induration_diameter,Result
0,1,0,22,2.25,14,0,0,1,51,50,1
1,1,0,15,3.00,2,0,0,1,900,70,1
2,1,0,16,10.50,2,1,0,0,100,25,1
3,1,0,27,4.50,9,0,0,1,80,30,1
4,1,0,20,8.00,6,1,0,0,45,8,1
...,...,...,...,...,...,...,...,...,...,...,...
85,1,0,40,5.50,8,0,0,1,69,5,1
86,1,0,38,7.50,8,0,1,0,56,45,1
87,1,0,46,11.50,4,1,0,0,91,25,0
88,1,0,32,12.00,9,1,0,0,43,50,0


# 3- Detecting Outliers


In [79]:
def find_outliers_tukey(x):
	q1 = np.percentile(x, 25)
	q3 = np.percentile(x, 75)
	iqr = q3-q1
	floor = q1 - 4*iqr
	ceiling = q3 + 4*iqr
	outlier_indices = list(x.index[(x < floor)|(x > ceiling)])
	outlier_values = list(x[outlier_indices])

	return outlier_indices, outlier_values

In [84]:
Age = df2.Age
Time = df2.Time
Nbr_Warts = df2.Nbr_Warts
Area = df2.Area
Induration_diameter = df2.Induration_diameter

In [85]:
tukey_indices, tukey_values = find_outliers_tukey(Area)

print("tukey_indices are :",tukey_indices)
print("tukey_values are :",tukey_values)

tukey_indices are : [1, 23, 34, 37, 40, 72, 75, 78]
tukey_values are : [900, 300, 502, 504, 352, 379, 367, 507]


In [86]:
tukey_indices, tukey_values = find_outliers_tukey(Induration_diameter)

print("tukey_indices are :",tukey_indices)
print("tukey_values are :",tukey_values)

tukey_indices are : [0, 1, 3, 26, 28, 30, 31, 33, 56, 58, 60, 61, 63, 86, 88]
tukey_values are : [50, 70, 30, 45, 50, 50, 70, 30, 45, 50, 50, 70, 30, 45, 50]
